In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from petitRADTRANS import Radtrans
from petitRADTRANS.retrieval import Retrieval, RetrievalConfig
from petitRADTRANS.retrieval.models import *
from petitRADTRANS.retrieval.chemistry import get_abundances
from petitRADTRANS.retrieval.parameter import Parameter
import petitRADTRANS.retrieval.cloud_cond as cc
from petitRADTRANS.retrieval.util import *

import scicomap as sc
import pandas as pd
import copy as cp
from glob import glob
import json

In [3]:
base_dir = "/Users/nasedkin/Documents/RetrievalResults/HR8799_2023_Retrievals/"
planet_dir = "e_full/"
retrieval_name = "HR8799e_23_v01_diseq_all_eqFeMgCloud_newphot_newSPH_fseds_mrprior"

with open(f"{base_dir}{planet_dir}out_PMN/{retrieval_name}_params.json") as paramfile:
    param_names = json.load(paramfile)

samples = pd.read_csv(f"{base_dir}{planet_dir}out_PMN/{retrieval_name}_post_equal_weights.dat",names=param_names,delim_whitespace=True,index_col=False)

/var/folders/zb/1kks1nb14h7_f6zq97b6sg8xxc6_b4/T/ipykernel_15537/4267690685.py:8: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  samples = pd.read_csv(f"{base_dir}{planet_dir}out_PMN/{retrieval_name}_post_equal_weights.dat",names=param_names,delim_whitespace=True,index_col=False)


In [6]:
parameters = {}
parameters['D_pl'] = Parameter('D_pl', False, value = 10.0*nc.pc)
parameters['mass'] = Parameter('mass',False,value = samples["mass"].median())
parameters['R_pl'] = Parameter('R_pl', False, value = samples["R_pl"].median())
   
parameters['T_int'] = Parameter('T_int', False, value = samples["T_int"].median())
parameters['T1'] = Parameter('T1', False, value = samples["T1"].median())
parameters['T2'] = Parameter('T2', False, value = samples["T2"].median())
parameters['T3'] = Parameter('T3', False, value = samples["T3"].median())
parameters['log_delta'] = Parameter('log_delta', False, value = samples["log_delta"].median())
parameters['alpha'] = Parameter('alpha', False, value = samples["alpha"].median())

parameters['Fe/H'] = Parameter('[Fe/H]', False, value = samples["Fe/H"].median())
parameters['C/O'] = Parameter('C/O', False, value = samples["C/O"].median())
parameters['log_pquench'] = Parameter('log_pquench', False, value = samples["log_pquench"].median())

parameters['sigma_lnorm'] = Parameter('sigma_lnorm', False, value = samples["sigma_lnorm"].median())
parameters['fsed_MgSiO3(c)'] = Parameter('fsed_MgSiO3(c)', False, value = samples["fsed_MgSiO3(c)"].median())
parameters['fsed_Fe(c)'] = Parameter('fsed_Fe(c)', False, value =  samples["fsed_Fe(c)"].median())
parameters['log_kzz'] = Parameter('log_kzz', False, value = samples["log_kzz"].median())

#parameters[ 'OSIRIS2011_scale_factor'] =\
#Parameter( 'OSIRIS2011_scale_factor', False, value = samples['OSIRIS2011_scale_factor'].median())

parameters['eq_scaling_MgSiO3(c)'] = Parameter('eq_scaling_MgSiO3(c)', False, value = samples["eq_scaling_MgSiO3(c)"].median())
#parameters['log_Pbase_MgSiO3(c)'] = Parameter('log_Pbase_MgSiO3(c)', False, value = samples["log_Pbase_MgSiO3(c)"].median())

parameters['eq_scaling_Fe(c)'] = Parameter('eq_scaling_Fe(c)', False, value = samples["eq_scaling_Fe(c)"].median())
#parameters['log_Pbase_Fe(c)'] = Parameter('log_Pbase_Fe(c)', False, value = samples["log_Pbase_Fe(c)"].median())
#print(parameters['log_kzz'].value,parameters['fsed'].value,parameters['sigma_lnorm'].value)

In [ ]:
line_species = ['H2O_Exomol_R_300', 
                'CO_all_iso_HITEMP_R_300', 
                'CH4_R_300', 
                'CO2_R_300', 
                'HCN_R_300', 
                'FeH_R_300', 
                'H2S_R_300', 
                'NH3_R_300', 
                'PH3_R_300', 
                'Na_allard_R_300', 
                'K_allard_R_300', 
                'TiO_all_Exomol_R_300', 
                'VO_R_300',
                'SiO_R_300',
                'FeH_R_300']

rayleigh_species = ['H2', 'He']
continuum_opacities = ['H2-H2', 'H2-He']
cloud_species = ['MgSiO3(c)_cd', 'Fe(c)_cd']

atmosphere = Radtrans(line_species = line_species,
                      rayleigh_species= rayleigh_species,
                      continuum_opacities = continuum_opacities,
                      cloud_species = cloud_species,
                      mode='c-k',
                      do_scat_emis=True,
                      wlen_bords_micron = [0.8,5.2],
                      hack_cloud_photospheric_tau = True)
pressures = np.logspace(-6,2,100)
atmosphere.setup_opa_structure(pressures)